We'll have to clone the git repositry for BERT to access the
 scripts for preprocessing our data for pretraining and also a script that runs further pre-training on a provided text corpus. The original BERT repo has been forked to my github where it was edited to remove the <i> next sentence prediction</i> task that BERT as it's redundant when processing tweets

In [0]:
!git clone https://github.com/fionnmcconville/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 354, done.
remote: Total 354 (delta 0), reused 0 (delta 0), pack-reused 354
Receiving objects: 100% (354/354), 295.74 KiB | 2.76 MiB/s, done.
Resolving deltas: 100% (196/196), done.


Importing dependencies...

In [3]:
import os
import pandas as pd
%tensorflow_version 1.x
import tensorflow as tf
import pprint
import re
import json
import tweepy
import html
!pip install contractions
import contractions


     |████████████████████████████████| 317kB 7.0MB/s 
     |████████████████████████████████| 245kB 51.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81706 sha256=b7dd51fa5335fc48744e5ebb061cd75c219de5cb0358f88a727a120791b8e626
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


**We'll use a TPU provided by google colab to run our model**

In [0]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

USE_TPU=True
try:
  # This address identifies the TPU we'll use when configuring TensorFlow.
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tf.config.experimental_connect_to_host(TPU_WORKER)
except Exception as ex:
  print(ex)
  USE_TPU=False

print("        USE_TPU:", USE_TPU)
print("Eager Execution:", tf.executing_eagerly())

assert not tf.executing_eagerly(), "Eager execution on TPUs have issues currently"

TPU address is grpc://10.126.13.50:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 14009160265551240099),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2764167678284809228),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8258336814111413720),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 7943687154834491057),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17352256950628607191),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 11333542380581171697),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 18276010718365558200),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2884050733167919613),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 151747856903

Set random seed and check tensorflow version

In [0]:
tf.set_random_seed(3060)
print("Tensorflow Version:", tf.__version__)

Tensorflow Version: 1.15.0


We'll put the further pre-trained model in a subdirectory inside our BERT model directory on our google bucket. There'll be functionality to delete the subdirectory we create when we make the BERT model, just in case things go wrong

In [0]:
#Bert uncased Large 
#bert_model_name = 'uncased_L-24_H-1024_A-16' 

#Large whole word masking
bert_model_name = 'wwm_uncased_L-24_H-1024_A-16' 

output_dir = \
os.path.join(bert_model_name, 'further_pretrained_model')

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = True #@param {type:"boolean"}
BUCKET = 'csc3002' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, output_dir)
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model *****


Upon research I found a favourable approach to further pre-training was to do within-task or in-domain pretraining with around 120,000 steps proven to be a success. I've already collected around 150,000 tweets that are categorized as either hate speech, offensive or benign so this roughly satisfies the requirement that the pretraining data is within-task or in-domain

In [0]:
!gcloud config set project 'my-project-csc3002'
!pip install gcsfs
data = pd.read_csv('gs://csc3002/Raw_Data/final.csv' , sep=',',  index_col = False, encoding = 'utf-8')
data.rename(columns={'Tweet': 'text'}, inplace = True)
print("\nThe amount of tweets in this dataframe is", len(data.index))
data.head()

Updated property [core/project].

The amount of tweets in this dataframe is 147977


,Hate_Speech,Offensive,text
0,0,1,@USER She should ask a few native Americans wh...
1,0,1,@USER @USER Go home you’re drunk!!! @USER #MAG...
2,0,0,Amazon is investigating Chinese employees who ...
3,0,1,"@USER Someone should'veTaken"" this piece of sh..."
4,0,0,@USER @USER Obama wanted liberals &amp; illega...


We just want the tweets from this file, in this stage of pre-training the other labels are meaningless as we're only performing masked language tasks and possibly next sentence prediction (although I'm not sure if next sentence prediction is striclty necessary....we'll see)

**Let's preprocess the pretraining data, identical to the pre-processing for our fine tuning data**

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive'
%pip install demoji-0.1.5-py3-none-any.whl

import demoji
demoji.download_codes()
def emojiReplace(text_string):
    
    emoji_dict = demoji.findall(text_string)
    for emoji in emoji_dict.keys():
        text_string = text_string.replace(emoji, ' '+  emoji_dict[emoji])
    return text_string

data['text'] = data['text'].apply(emojiReplace)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive
... OK (Got response in 0.10 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [0]:
def preprocess(text_string):
    """
    Accepts a text string and:
    1) Removes URLS
    2) lots of whitespace with one instance
    3) Removes mentions
    4) Uses the html.unescape() method to convert unicode to text counterpart
    5) Replace & with and
    6) Remove the fact the tweet is a retweet if it is - knowing the tweet is 
       a retweet does not help towards our classification task.
    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[#$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+:'
    mention_regex1 = '@[\w\-]+'
    RT_regex = '(RT|rt)[ ]*@[ ]*[\S]+'
    
    # Replaces urls with URL
    parsed_text = re.sub(giant_url_regex, '', text_string)
    parsed_text = re.sub('URL', '', parsed_text)
    
    # Remove the fact the tweet is a retweet. 
    # (we're only interested in the language of the tweet here)
    parsed_text = re.sub(RT_regex, ' ', parsed_text) 
    
    # Removes mentions as they're redundant information
    parsed_text = re.sub(mention_regex, '',  parsed_text)
    #including mentions with colons after - this seems to come up often
    parsed_text = re.sub(mention_regex1, '',  parsed_text)  

    # Remove unicode
    parsed_text = re.sub(r'[^\x00-\x7F]','', parsed_text) 
    parsed_text = re.sub(r'&#[0-9]+;', '', parsed_text)  

    # Convert unicode missed by regex to text
    parsed_text = html.unescape(parsed_text)

    # Remove excess whitespace at the end
    parsed_text = re.sub(space_pattern, ' ', parsed_text) 

    #Replace &amp; with and
    parsed_text = re.sub('&amp;', 'and', parsed_text)
    
    # Set text to lowercase and strip
    parsed_text = parsed_text.lower()
    parsed_text = parsed_text.strip()

    # Replace contractions with their full worded form
    parsed_text = contractions.fix(parsed_text)
    
    return parsed_text

Hashtag Segmentation

In [0]:
!pip install wordsegment
import wordsegment as ws
from wordsegment import load, segment

load()
#The values below of the bigrams reflect the amount of search results on google that come up
ws.BIGRAMS['alt right'] = 1.17e8 # update wordsegment dict so it recognises altright as "alt right" rather than salt right
ws.BIGRAMS['white supremacists'] = 3.86e6
ws.BIGRAMS['tweets'] = 6.26e10

def hashtagSegment(text_string):
    
    #We target hashtags so that we only segment the hashtag strings.
    #Otherwise the segment function may operate on misspelled words also; which
    #often appear in hate speech tweets owing to the ill education of those spewing it
    temp_str = []
    for word in text_string.split(' '):
        if word.startswith('#') == False:
            temp_str.append(word)
        else:
            temp_str = temp_str + segment(word)
            
    text_string = ' '.join(temp_str)       
    return text_string

data['text'] = data['text'].apply(hashtagSegment)

In [0]:
bucket_dir = 'gs://csc3002'

bert_ckpt_dir = os.path.join(bucket_dir, bert_model_name) 

bert_ckpt_file   = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")
vocab_file = os.path.join(bert_ckpt_dir, "vocab.txt")

print("Using BERT checkpoint from:", bert_ckpt_dir)

Using BERT checkpoint from: gs://csc3002/wwm_uncased_L-24_H-1024_A-16


### Loading in the rest of the pretraining tweets

See RetrievingPretrainingData.ipynb for more details on the sources of these tweets and how they were pre-processed

In [0]:
imTweets = pd.read_csv('gs://csc3002/pretrain_data/tweetText/removedTerms/immigrationTweets.csv',\
                       sep=',',  index_col = False, names = ['id', 'text'])

womTweets = pd.read_csv('gs://csc3002/pretrain_data/tweetText/removedTerms/YesAllWomenTweets.csv',\
                        sep=',',  index_col = False, names =['id', 'text'])

sexismTweets = pd.read_csv('gs://csc3002/pretrain_data/tweetText/removedTerms/sexismTweets.csv',\
                           sep=',',  index_col = False, names =['id', 'text'])

chalk = pd.read_csv('gs://csc3002/pretrain_data/tweetText/removedTerms/chalkTweets.csv', \
                    sep=',',  index_col = False, names =['id', 'text'])

dat = pd.read_csv('gs://csc3002/pretrain_data/tweetText/removedTerms/full.csv',\
                  sep=',',  index_col = False, names =['id', 'text'])

dfs = [imTweets, womTweets, sexismTweets, chalk, dat, data]
full = pd.concat(dfs, axis =0)

full.drop_duplicates(subset='id',inplace =True ) #Important to drop duplicates
full.drop_duplicates(subset='text',inplace =True ) #Important to drop duplicates

#Shuffle Data
full = full.sample(frac=1)
full.reset_index(drop = True, inplace = True)

full.text = full.text.astype(str) #Making sure text column is type string so preprocessing works

full.drop(columns = {'Hate_Speech', 'Offensive'}, inplace=True)
#Lets pre-process one more time just to make sure
full['text'] = full['text'].apply(preprocess)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [0]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021852 entries, 0 to 1021851
Data columns (total 2 columns):
id      1021851 non-null float64
text    1021852 non-null object
dtypes: float64(1), object(1)
memory usage: 15.6+ MB


We can train BERT on a max of 512 sequence length. Having a large sequence length exponentially increases the amount of memory required to process tensors, and thus increases our time training. 

<i>(Instead of just removing tweets under our defined max seq length we could run the portion of the dataset with a large sequence length on the last tenth of the amount of steps. Further investigation needed here though)</i>

In [0]:
#We'll see if we can get away with max seq length of 256 first
MAX_SEQ_LEN = 256

print("There will be", len(full[full['text'].apply(lambda x: len(x) < MAX_SEQ_LEN)]), "tweets")

#The tweets are from many different sources and have been grouped together 
#so we'll shuffle the data
full = full.sample(frac=1)
full.reset_index(drop = True, inplace=True)
full = full[full['text'].apply(lambda x: len(x) <= MAX_SEQ_LEN)]

There will be 1021852 tweets


Lets save our data in a text file and put it in the bert repo directory we cloned

In [0]:
# Go back to Original environment space
%cd ../.. 
%cd bert/
tweets = full.text
tweets.to_csv('./text_file.txt', sep=',', index = True)
%ls

/content
/content/bert


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.


bert/
CONTRIBUTING.md
create_pretraining_data.py
extract_features.py
__init__.py
LICENSE
modeling.py
modeling_test.py
multilingual.md
optimization.py
optimization_test.py
predicting_movie_reviews_with_bert_on_tf_hub.ipynb
__pycache__/
README.md
requirements.txt
run_classifier.py
run_classifier_with_tfhub.py
run_pretraining.py
run_squad.py
sample_text.txt
text_file.txt
tokenization.py
tokenization_test.py


I believe the exact string and integer values have to be passed to both scripts

In [0]:
#It's advised to set max predictions per sequence to around max_seq_length * 0.15
max_preds_per_seq = MAX_SEQ_LEN * 0.15
print("max_predictions_per_seq:", max_preds_per_seq)

print("vocab file location: ", vocab_file)

output_file = os.path.join(OUTPUT_DIR, 'pretrainingdata.tfrecord' )
print("output file", output_file)

max_predictions_per_seq: 38.4
vocab file location:  gs://csc3002/wwm_uncased_L-24_H-1024_A-16/vocab.txt
output file gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model/pretrainingdata.tfrecord


In [0]:
!python create_pretraining_data.py \
  --input_file='./text_file.txt' \
  --output_file='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model/pretrainingdata.tfrecord' \
  --vocab_file='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/vocab.txt' \
  --do_lower_case=True \
  --do_whole_word_mask =True \
  --max_seq_length=256 \
  --max_predictions_per_seq=39 \
  --masked_lm_prob=0.15 \
  --random_seed=3060 \
  --dupe_factor=5 \
  --short_seq_prob=0.25

  #The output is a set of tf.train.Examples serialized into TFRecord file format



W0205 14:11:21.874597 139984891434880 module_wrapper.py:139] From create_pretraining_data.py:440: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0205 14:11:21.874871 139984891434880 module_wrapper.py:139] From create_pretraining_data.py:440: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0205 14:11:21.875098 139984891434880 module_wrapper.py:139] From /content/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0205 14:11:24.395494 139984891434880 module_wrapper.py:139] From create_pretraining_data.py:447: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0205 14:11:24.397476 139984891434880 module_wrapper.py:139] From create_pretraining_data.py:449: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:*** Reading from input files ***
I0205 14:11:24.3977

Now we run the main pretraining script, which creates the further pretrained model

In [0]:
print("Input file:", OUTPUT_DIR )
print("Bert checkpoint file location:", bert_ckpt_file )
print("Bert config file location:", bert_config_file )
print("TPU ADDRESS: ", TPU_ADDRESS)

Input file: gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model
Bert checkpoint file location: gs://csc3002/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt
Bert config file location: gs://csc3002/wwm_uncased_L-24_H-1024_A-16/bert_config.json
TPU ADDRESS:  grpc://10.126.13.50:8470


In [0]:
#TPU address changes each new runtime. Change to the value above to ensure run success
!python run_pretraining.py \
  --input_file='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model/pretrainingdata.tfrecord' \
  --output_dir='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model'\
  --do_train=True \
  --do_eval=True \
  --bert_config_file='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/bert_config.json' \
  --init_checkpoint='gs://csc3002/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt' \
  --train_batch_size=32 \
  --max_seq_length=256 \
  --save_checkpoints_steps=20000 \
  --max_predictions_per_seq=39 \
  --num_train_steps=120000 \
  --num_warmup_steps=12000 \
  --learning_rate=5e-5\
  --use_tpu=True \
  --tpu_name='grpc://10.126.13.50:8470'\
  --num_tpu_cores=8




W0205 15:37:09.464732 140633744410496 module_wrapper.py:139] From run_pretraining.py:409: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0205 15:37:09.464992 140633744410496 module_wrapper.py:139] From run_pretraining.py:409: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0205 15:37:09.465197 140633744410496 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0205 15:37:11.075771 140633744410496 module_wrapper.py:139] From run_pretraining.py:416: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0205 15:37:11.361335 140633744410496 module_wrapper.py:139] From run_pretraining.py:420: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0205 15:37:11.491924 140633744410496 module_wrapper.py:139] From run_pretraining.py:422: The name tf.logg

# We can use tensorboard to visualise the performance of our model

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip   #Downloads file to google drive
def get_tensorboard(path_to_event_file):
  get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 --reload_multifile=true &'
.format(path_to_event_file))
  
  get_ipython().system_raw('./ngrok http 6006 &')

  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

get_tensorboard('gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model') #gs://csc3002/wwm_uncased_L-24_H-1024_A-16/further_pretrained_model

--2020-02-06 11:49:25--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.206.168.246, 34.204.59.252, 52.20.12.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.206.168.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.13M  18.5MB/s    in 0.7s    

2020-02-06 11:49:26 (18.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: ngrok                   
https://95013a55.ngrok.io
